In [14]:
import numpy as np
import pickle

class rating:
    
    def __init__(self):
        # weights for dimension
        self.dimension = [[0.21, 0.12, 0.25, 0.18, 0.12, 0.12]]
        # weights for dimension1's indicators
        self.indicator1 = [
            [1, 1, 1, 0.6, 0.4, 1, 1, 1, 1, 1],
            [0.7, 0.3, 0.7, 0.3, 0.7, 0.3, 0.3, 0.3, 0.4],
            [0.29, 0.19, 0.23, 0.29]
        ]
        # weights for dimension2's indicators
        self.indicator2 = [
            [0.4, 0.6, 0.4, 0.6, 0.06, 0.25, 0.22, 0.19, 0.11, 0.12, 0.05, 1, 1, 1],
            [0.16, 0.25, 0.18, 0.13, 0.15, 0.13]
        ]
        # weights for dimension3's indicators
        self.indicator3 = [
            [0.3, 0.7, 0.34, 0.31, 0.35, 0.31, 0.31, 0.38],
            [0.41, 0.35, 0.24]
        ]
        # weights for dimension4's indicators
        self.indicator4 = [[0.25, 0.21, 0.15, 0.21, 0.18]]
        # weights for dimension5's indicators
        self.indicator5 = [[0.55, 0.45]]
        # weights for dimension6's indicators
        self.indicator6 = [[0.50, 0.50]]
        
        # combine weights together
        self.indicators = [
            self.indicator1, self.indicator2, self.indicator3, self.indicator4, self.indicator5, self.indicator6
        ]
        # calculate the number of factors for each dimension
        self.num_factors = [len(item[0]) for item in self.indicators]
        
    def preprocess(self, inputs_list_one_by_one):
        inputs, start = [], 0
        for num in self.num_factors:
            inputs.append(inputs_list_one_by_one[start : start+num])
            start += num
        return inputs
    
    def score(self, weights, num_input):
        w_sum = 0
        ws = []
        inputs = []
        scores = []
        for i, w in enumerate(weights):
            if w == 1:
                score = w * num_input[i]
            else:
                w_sum += w
                ws.append(w)
                inputs.append(num_input[i])
                if w_sum == 1:
                    score = np.sum(np.array(ws) * np.array(inputs))
                    w_sum = 0
                    ws = []
                    inputs = []
                else:
                    continue
            scores.append(score)
        return scores
    
    def score_dimension(self, dimension, num_input):
        for item in dimension:
            if item:
                score = self.score(item, num_input)
                num_input = score
        return round(score[0], 2)
    
    def predict(self, inputs):
        if len(inputs) > 6:
            inputs = self.preprocess(inputs)
        else:
            inputs = inputs
        scores_dim = list(map(self.score_dimension, self.indicators, inputs))
        scores_overall = self.score_dimension(self.dimension, scores_dim)
        return scores_dim, scores_overall
    
    
if __name__ == "__main__":
    # inputs
    input1 = [0.4, 0.5, 0.8, 0.9, 0.1, 0.1, 0.05, 0.9, 0.9, 0.9]
    input2 = [0.2, 0.5, 0.9, 0.1, 1, 1, 1, 1, 0, 0, 0, 0.8, 0.8, 0.9]
    input3 = [0.4, 0.3, 0.5, 0.7, 0.9, 0.1, 0.2, 0.9]
    input4 = [0.8, 0.7, 0.6, 0.5, 1]
    input5 = [0.8, 0.8]
    input6 = [0.7, 0.6]
    inputs = [input1, input2, input3, input4, input5, input6]

    # rating system parameters
    mod = rating()

    # score for each dim and for overall
    score_dim, score_overall = mod.predict(inputs_list)
    # save model to .pkl file
    fn = "rating.pkl"
    with open(fn, "wb") as f:
        pickle.dump(mod, f)
    print("Model dumped!")
    
    # load model from .pkl file
    mod = pickle.load(open("rating.pkl", "rb"))

Model dumped!


In [1]:
from flask import Flask, request, jsonify
import pickle
import numpy as np
import traceback
import sys
from rating import rating

# API definition
app = Flask(__name__)

@app.route("/predict", methods = ["POST"])
def predict():
    if mod:
        json_ = request.json
        query = list(json_[0].values())
        dim_pred, overall_pred = mod.predict(query)
        return jsonify({"score_dimension": dim_pred, "score_overall": overall_pred})
    else:
        print("Train the model first")
        return("No model here to use")

if __name__ == "__main__":
    try:
        port = int(sys.argv[1])
    except:
        port = 12345
    mod = pickle.load(open("rating.pkl", "rb"))
    print("Model loaded")
    
    app.run(port = port, debug = True)

Model loaded
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with windowsapi reloader


SystemExit: 1

C:\Users\Cheytac\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
%%writefile pre.py

import numpy as np
from flask import Flask, render_template, request
import pickle
from rating import rating

# Initialize the flask APP
app = Flask(__name__)
# Load the model
mod = pickle.load(open("rating.pkl", "rb"))

# Default page for our web-app
@app.route("/")
def home():
    return render_template("index.html")

# Use the predict button in our web-app
@app.route("/predict", methods = ["POST"])
def predict():
    # For rendering results on HTML GUI
    features = [float(x) for x in request.form.values()]
    score_dim, score_overall = mod.predict(features)
    return render_template("index.html", prediction_text = "The overall score for the shop is: {}".format(score_overall))

if __name__ == "__main__":
    app.run(debug = True)


Writing pre.py


In [3]:
%%writefile index.html

<!DOCTYPE html>
<html >

<head>
  <meta charset="UTF-8">
  <title>Shop Rating Predictor</title>
  <link href='https://fonts.googleapis.com/css?family=Pacifico' rel='stylesheet' type='text/css'>
<link href='https://fonts.googleapis.com/css?family=Arimo' rel='stylesheet' type='text/css'>
<link href='https://fonts.googleapis.com/css?family=Hind:300' rel='stylesheet' type='text/css'>
<link href='https://fonts.googleapis.com/css?family=Open+Sans+Condensed:300' rel='stylesheet' type='text/css'>
<link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">

</head>

<body>
 <div class="login">
    <h1>Predict Shop Rating</h1>
     <h3> Enter the following values to predict the shop rating</h3>

     <!-- Main Input For Receiving Query to our ML -->
    <form action="{{ url_for('predict')}}"method="post">
        <input type="text" name="factor1" placeholder="Factor 1" required="required" />
        <input type="text" name="factor2" placeholder="Factor 2" required="required" />
        <input type="text" name="factor3" placeholder="Factor 3" required="required" />
        <input type="text" name="factor4" placeholder="Factor 4" required="required" />
        <input type="text" name="factor5" placeholder="Factor 5" required="required" />
        <input type="text" name="factor6" placeholder="Factor 6" required="required" />
        
        <button type="submit" class="btn">Predict</button>
    </form>
    
    <br>
    <br>
    {{ prediction_text }}

</div>

</body>
</html>

Overwriting index.html


In [4]:
%%writefile style.css

@import url(https://fonts.googleapis.com/css?family=Open+Sans);

.btn{
  background-color: #ba0b0b; /* Green */
  border: none;
  text-align: center;
  border-radius: 8px;
  padding: 14px 40px;
  text-decoration: none;
  display: inline-block;
  font-size: 20px;
  margin: 4px 2px;
  cursor: pointer;
}
.btn:hover { background-color: #0a0000; box-shadow: 0 12px 16px 0 rgba(0,0,0,0.24),0 17px 50px 0 rgba(0,0,0,0.19); }

* { -webkit-box-sizing:border-box; -moz-box-sizing:border-box; -ms-box-sizing:border-box; -o-box-sizing:border-box; box-sizing:border-box; }

html { width: 100%; height:100%; overflow:hidden; }

body {
	width: 100%;
	height:100%;
	font-family: 'Open Sans', sans-serif;
	background: #331952;
	color: #fff;
	font-size: 18px;
	text-align:center;
	letter-spacing:1.2px;
	background: -moz-radial-gradient(0% 100%, ellipse cover, rgba(104,128,138,.4) 10%,rgba(138,114,76,0) 40%),-moz-linear-gradient(top,  rgba(57,173,219,.25) 0%, rgba(42,60,87,.4) 100%), -moz-linear-gradient(-45deg,  #670d10 0%, #092756 100%);
	background: -webkit-radial-gradient(0% 100%, ellipse cover, rgba(104,128,138,.4) 10%,rgba(138,114,76,0) 40%), -webkit-linear-gradient(top,  rgba(57,173,219,.25) 0%,rgba(42,60,87,.4) 100%), -webkit-linear-gradient(-45deg,  #670d10 0%,#092756 100%);
	background: -o-radial-gradient(0% 100%, ellipse cover, rgba(104,128,138,.4) 10%,rgba(138,114,76,0) 40%), -o-linear-gradient(top,  rgba(57,173,219,.25) 0%,rgba(42,60,87,.4) 100%), -o-linear-gradient(-45deg,  #670d10 0%,#092756 100%);
	background: -ms-radial-gradient(0% 100%, ellipse cover, rgba(104,128,138,.4) 10%,rgba(138,114,76,0) 40%), -ms-linear-gradient(top,  rgba(57,173,219,.25) 0%,rgba(42,60,87,.4) 100%), -ms-linear-gradient(-45deg,  #670d10 0%,#092756 100%);
	background: -webkit-radial-gradient(0% 100%, ellipse cover, rgba(104,128,138,.4) 10%,rgba(138,114,76,0) 40%), linear-gradient(to bottom,  rgba(57,173,219,.25) 0%,rgba(42,60,87,.4) 100%), linear-gradient(135deg,  #670d10 0%,#092756 100%);
	filter: progid:DXImageTransform.Microsoft.gradient( startColorstr='#3E1D6D', endColorstr='#092756',GradientType=1 );

}
.login {
	position: absolute;
	top: 40%;
	left: 50%;
	margin: -150px 0 0 -150px;
	width:400px;
	height:400px;
}

.login h1 { color: #fff; text-shadow: 0 0 10px rgba(0,0,0,0.3); letter-spacing:1px; text-align:center; }

input {
	width: 100%;
	margin-bottom: 10px;
	background: rgba(255,0,0,0.3);
	border: none;
	outline: none;
	padding: 10px;
	font-size: 13px;
	color: #fff;
	text-shadow: 1px 1px 1px rgba(0,0,0,0.3);
	border: 1px solid rgba(0,0,0,0.3);
	border-radius: 4px;
	box-shadow: inset 0 -5px 45px rgba(100,100,100,0.2), 0 1px 1px rgba(255,255,255,0.2);
	-webkit-transition: box-shadow .5s ease;
	-moz-transition: box-shadow .5s ease;
	-o-transition: box-shadow .5s ease;
	-ms-transition: box-shadow .5s ease;
	transition: box-shadow .5s ease;
}
input:focus { box-shadow: inset 0 -5px 45px rgba(100,100,100,0.4), 0 1px 1px rgba(255,255,255,0.2); }

Writing style.css
